In [1]:
import keras
import kapre
from datetime import datetime
now = datetime.now()

def print_version_info():
    print('%s/%s/%s' % (now.year, now.month, now.day))
    print('Keras version: {}'.format(keras.__version__))
    if keras.backend._BACKEND == 'tensorflow':
        import tensorflow
        print('Keras backend: {}: {}'.format(keras.backend._backend, tensorflow.__version__))
    else:
        import theano
        print('Keras backend: {}: {}'.format(keras.backend._backend, theano.__version__))
    print('Keras image dim ordering: {}'.format(keras.backend.image_dim_ordering()))
    print('Kapre version: {}'.format(kapre.__version__))

def check_model(model):
    model.summary(line_length=80, positions=[.33, .65, .8, 1.])

    batch_input_shape = (2,) + model.input_shape[1:]
    batch_output_shape = (2,) + model.output_shape[1:]
    model.compile('sgd', 'mse')
    model.fit(np.random.uniform(size=batch_input_shape), np.random.uniform(size=batch_output_shape), epochs=1)

Couldn't import dot_parser, loading of dot files will not be possible.


Using TensorFlow backend.


# Spectrogram

In [2]:
print_version_info()
import keras
import kapre
from keras.models import Sequential
from kapre.time_frequency import Spectrogram
import numpy as np
src = np.random.random((2, 44100))
sr = 44100
model = Sequential()
model.add(Spectrogram(n_dft=512, n_hop=256, input_shape=src.shape, 
          return_decibel_spectrogram=True, power_spectrogram=2.0, 
          trainable_kernel=False, name='static_stft'))

check_model(model)

2017/3/16
Keras version: 2.0.0
Keras backend: tensorflow: 1.0.1
Keras image dim ordering: tf
Kapre version: 0.0.3.1
________________________________________________________________________________
Layer (type)              Output Shape              Param #     
static_stft (Spectrogram) (None, 257, 173, 2)       263168      
Total params: 263,168.0
Trainable params: 0.0
Non-trainable params: 263,168
________________________________________________________________________________
Epoch 1/1
2/2 [==============================] - 0s - loss: 1167.1609


# Spectrogram (trainable)

In [3]:
print_version_info()
import keras
import kapre
from keras.models import Sequential
from kapre.time_frequency import Spectrogram
import numpy as np

src = np.random.random((2, 44100))
sr = 44100
model = Sequential()
model.add(Spectrogram(n_dft=2048, n_hop=1024, input_shape=src.shape, 
          return_decibel_spectrogram=True, power_spectrogram=2.0, 
          trainable_kernel=True, name='trainable_stft'))

check_model(model)

2017/3/16
Keras version: 2.0.0
Keras backend: tensorflow: 1.0.1
Keras image dim ordering: tf
Kapre version: 0.0.3.1
________________________________________________________________________________
Layer (type)              Output Shape              Param #     
trainable_stft (Spectrogr (None, 1025, 44, 2)       4198400     
Total params: 4,198,400.0
Trainable params: 4,198,400
Non-trainable params: 0.0
________________________________________________________________________________
Epoch 1/1
2/2 [==============================] - 0s - loss: 1594.0073


# Melspectrogram

In [4]:
print_version_info()
import keras
import kapre
from keras.models import Sequential
from kapre.time_frequency import Melspectrogram
import numpy as np

src = np.random.random((2, 44100))
sr = 44100
model = Sequential()
model.add(Melspectrogram(sr=16000, n_mels=128, 
          n_dft=512, n_hop=256, input_shape=src.shape, 
          return_decibel_spectrogram=True,
          trainable_kernel=True, name='melgram'))

check_model(model)

2017/3/16
Keras version: 2.0.0
Keras backend: tensorflow: 1.0.1
Keras image dim ordering: tf
Kapre version: 0.0.3.1
________________________________________________________________________________
Layer (type)              Output Shape              Param #     
melgram (Melspectrogram)  (None, 128, 173, 2)       296064      
Total params: 296,064
Trainable params: 263,168
Non-trainable params: 32,896
________________________________________________________________________________
Epoch 1/1
2/2 [==============================] - 0s - loss: 0.3335


# AdditiveNoise

In [5]:
print_version_info()
import keras
import kapre
from keras.models import Sequential
from kapre.time_frequency import Melspectrogram
from kapre.augmentation import AdditiveNoise
import numpy as np

src = np.random.random((2, 44100))
sr = 44100
model = Sequential()
model.add(Melspectrogram(sr=16000, n_mels=128, 
          n_dft=512, n_hop=256, input_shape=src.shape, 
          return_decibel_spectrogram=True,
          trainable_kernel=False, name='melgram'))
model.add(AdditiveNoise(power=0.2))
model.summary(line_length=80, positions=[.33, .65, .8, 1.])

2017/3/16
Keras version: 2.0.0
Keras backend: tensorflow: 1.0.1
Keras image dim ordering: tf
Kapre version: 0.0.3.1
________________________________________________________________________________
Layer (type)              Output Shape              Param #     
melgram (Melspectrogram)  (None, 128, 173, 2)       296064      
________________________________________________________________________________
additive_noise_1 (Additiv (None, 128, 173, 2)       0           
Total params: 296,064.0
Trainable params: 0.0
Non-trainable params: 296,064.0
________________________________________________________________________________


# Stft

In [6]:
import keras
import kapre
from keras.models import Sequential
from kapre.stft import Stft
import numpy as np

print('Keras version: {}'.format(keras.__version__))
print('Keras backend: {}'.format(keras.backend._backend))
print('Keras image dim ordering: {}'.format(keras.backend.image_dim_ordering()))
print('Kapre version: {}'.format(kapre.__version__))

src = np.random.random((2, 44100))
sr = 44100
model = Sequential()
model.add(Stft(n_fft=256, n_hop=64, return_decibel_stft=True,
              input_shape=src.shape))
model.summary(line_length=80, positions=[.33, .65, .8, 1.])

Keras version: 2.0.0
Keras backend: tensorflow
Keras image dim ordering: tf
Kapre version: 0.0.3.1


AsTensorError: ('Cannot convert Tensor("stft_1/mul:0", shape=(?, 2, 256), dtype=float32) to TensorType', <class 'tensorflow.python.framework.ops.Tensor'>)